In [1]:
from tqdm import tqdm 
import pickle
from joblib import Parallel, delayed

# Ensure the plotting backend is set correctly
%matplotlib inline

In [2]:
backup_file = 'data/grouped_dfs.pkl'
try:
    with open(backup_file, 'rb') as f:
        grouped_dfs = pickle.load(f)
except FileNotFoundError:
    print('Backup file not found. Rebuilding the data.')

# Spot MB

### Bottom Bounce

In [ ]:
# RIOT from Dec 19 2022 to Jan 17 2023
df_riot = grouped_dfs['RIOT']['2022-12-19':'2023-01-23']  # b/o on Jan 04, 23

In [ ]:
# INDI from Dec 19 2022 to Jan 23 2023
df_indi = grouped_dfs['INDI']['2022-12-19':'2023-01-23']
df_indi

In [ ]:
# PSFE from Dec 10 2022 to Jan 23 2023
df_psfe = grouped_dfs['PSFE']['2022-12-10':'2023-01-23']

### Consolidation Breakout
- Consolidation from 1 month to a year
- high relative volume (3x or more avg volume)
- stock will close near high
- 

In [ ]:
df_amkr = grouped_dfs['AMKR']['2022-12-21':'2023-01-23']
df_ever = grouped_dfs['EVER']['2022-12-01':'2023-01-10']

### Continuation Setup
- Focus on the 2nd or 3rd breakout
- Stock close near the high
- Day before the breakout, negative or small up day
- 3 to 10 days
- Not Up 2 days in the row
- Highest probability of success
- 2lynch

In [4]:
df_wwww = grouped_dfs['WWW']['2022-12-03':'2023-02-21']
df_wwww.to_csv('data/wwww.csv')

In [ ]:
df_amgn = grouped_dfs['AMGN']['2022-10-17':'2023-11-14']
df_tree = grouped_dfs['TREE']['2022-12-12':'2023-02-27']
df_fslr = grouped_dfs['FSLR']['2022-10-03':'2023-01-01']


#### 2 Lynch

In [ ]:
mb_tests = {
    'RIOT': df_riot,
    'INDI': df_indi,
    'PSFE': df_psfe,
    'AMKR': df_amkr,
    'EVER': df_ever,
    'AMGN': df_amgn,
    'TREE': df_tree,
    'FSLR': df_fslr
}

In [ ]:
import numpy as np

# Adjusted threshold based on observed range conditions
consolidation_period = 5  # Number of days in the narrow range
breakout_threshold = 0.2  # 20% breakout threshold, adjusted for better detection
volume_multiplier = 1.5  # 50% increase in volume

def detect_momentum_burst(df):
    bursts = []
    
    for i in range(consolidation_period, len(df)):
        # Identify consolidation (narrow range, low volume)
        recent_data = df.iloc[i-consolidation_period:i]
        range_condition = recent_data['High'].max() - recent_data['Low'].min()
        volume_condition = recent_data['Volume'].mean()

        if range_condition < recent_data['Close'].mean() * breakout_threshold:
            # Identify breakout (price increase, volume surge)
            if df['Close'].iloc[i] > recent_data['High'].max() and df['Volume'].iloc[i] > volume_condition * volume_multiplier:
                # Calculate 3-day and 5-day percentage gains
                if i + 3 < len(df):
                    gain_3d = (df['Close'].iloc[i + 3] - df['Close'].iloc[i]) / df['Close'].iloc[i] * 100
                else:
                    gain_3d = np.nan  # Not enough data to calculate 3-day gain

                if i + 5 < len(df):
                    gain_5d = (df['Close'].iloc[i + 5] - df['Close'].iloc[i]) / df['Close'].iloc[i] * 100
                else:
                    gain_5d = np.nan  # Not enough data to calculate 5-day gain

                burst = {
                    'date': df.index[i],
                    'close': df['Close'].iloc[i],
                    'high': df['High'].iloc[i],
                    'low': df['Low'].iloc[i],
                    'open': df['Open'].iloc[i],
                    'volume': df['Volume'].iloc[i],
                    'breakout_percentage': (df['Close'].iloc[i] - df['Close'].iloc[i-1]) / df['Close'].iloc[i-1] * 100,
                    'gain_3d': gain_3d,
                    'gain_5d': gain_5d
                }
                bursts.append(burst)
    
    return bursts


# Parallel processing function with progress bar
def process_symbol(symbol, df):
    return symbol, detect_momentum_burst(df)

# Apply the function to all dataframes in parallel with tqdm progress bar
all_bursts = dict(Parallel(n_jobs=-1)(delayed(process_symbol)(symbol, df) for symbol, df in tqdm(mb_tests.items(), total=len(mb_tests))))
all_bursts = {symbol: bursts for symbol, bursts in all_bursts.items() if bursts}
len(all_bursts)

In [ ]:
all_bursts